## Crossword Generation using BlackSquare - Proto

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import puz
import unicodedata

from blacksquare import Crossword, BLACK, EMPTY, ACROSS, DOWN, DEFAULT_WORDLIST
from IPython.display import HTML

In [2]:
xw = Crossword(7, display_size_px = 200)
# xw[3, 3] = BLACK
xw[0, 0] = BLACK
xw[0, 1] = BLACK
xw[1, 0] = BLACK
xw.fill()

Output()

,,1 B,2 C,3 C,4 M,5 E
,6 P,E,R,S,E,S
7 S,O,N,A,T,A,S
8 H,O,T,Y,O,G,A
9 O,H,S,O,R,R,Y
10 L,E,E,L,E,E,
11 E,D,N,A,S,,


In [7]:
xw = Crossword(19, display_size_px = 600)
black_cells = [[0, 3], [0, 9], [0, 10], [0, 11], [0, 15], [1, 3], [1, 10], [2, 3], [3, 7], [3, 14], [3, 18], [4, 4], [4, 5], [4, 12], [4, 13], [5, 4], [5, 10], [5, 11], [5, 12], [6, 0], [6, 6], [6, 10], [6, 11], [6, 12], [7, 6], [7, 7], [7, 8], [7, 9], [7, 16], [7, 17], [7, 18], [8, 7], [8, 8], [8, 15], [9, 4], [9, 5], [9, 13], [9, 14], [10, 3], [10, 10], [10, 11], [11, 0], [11, 1], [11, 2], [11, 9], [11, 10], [11, 11], [11, 12], [12, 6], [12, 7], [12, 8], [12, 12], [12, 18], [13, 6], [13, 7], [13, 8], [13, 14], [14, 5], [14, 6], [14, 13], [14, 14], [15, 0], [15, 4], [15, 11], [16, 15], [17, 8], [17, 15], [18, 3], [18, 7], [18, 8], [18, 9], [18, 15]]

for cell in black_cells:
    xw[cell[0], cell[1]] = BLACK
    
grid = xw.fill(temperature = 1)

# display the grid output
display(HTML(grid._grid_html()))
json_data = grid.get_json_data()

Output()

In [ ]:
xw._grid

In [ ]:
lines = open("./blacksquare/xwordlist.dict").read().splitlines()
all_answers_list = []
for line in lines:
    answer = line.split(';')[0].lower()
    if not any(char.isdigit() for char in answer):
        all_answers_list.append(answer)

#### Intersection between 'xwordlist' and 'our answer list'

In [ ]:
our_answer_list = open('./all_answers.txt').read().splitlines()

set_1 = set(our_answer_list)
set_2 = set(all_answers_list)

print(f"Intersection between two answer sets: {len(set_1.intersection(set_2))}")
print(f"Union between two answer sets: {len(set_1.union(set_2))}")

#### Answer Length Histogram Plot in 'Xwordlist'

In [ ]:
answer_len_dict = {}
for answer in all_answers_list:
    if len(answer) in answer_len_dict.keys():
        answer_len_dict[len(answer)] += 1
    else:
        answer_len_dict[len(answer)] = 0

array_2d = []
for answer_len, count in answer_len_dict.items():
    if answer_len < 25 and answer_len > 2:
        array_2d.append([answer_len, count])
    
array_2d.sort(key = lambda x: x[0])

array_2d = np.array(array_2d)

plt.figure(figsize = (10, 4))
plt.bar(array_2d[:, 0], array_2d[:, 1])
plt.xticks(array_2d[:, 0])
plt.show()

#### Handling Symmetric Crosswords

In [ ]:
from blacksquare import Crossword, BLACK, EMPTY, ACROSS, DOWN, DEFAULT_WORDLIST, WordIndex, WordList
from tqdm import tqdm
class SymmetricCrossword(Crossword):
    def set_word(self, word_index: WordIndex, value: str) -> None:
        super().set_word(word_index, value)
        super().set_word(self.get_symmetric_word_index(word_index), value[::-1])

emordilaps = {}
for word, score in tqdm(DEFAULT_WORDLIST):
    reverse_score = DEFAULT_WORDLIST.get_score(word[::-1])
    if reverse_score:
        emordilaps[word] = min(score, reverse_score)
emordilaps_wordlist = WordList(emordilaps)

# Now just construct the puzzle and fill!
xw = SymmetricCrossword(7, display_size_px = 200)

filled = [
    (0, 5), (0, 6), (1, 6), (3, 3)
]

# filled = [
#  (0, 4), (0, 5), (0, 11), (1, 4), (1, 5), (1, 11),
#     (2, 4), (2, 11), (3, 4), (3, 9), (4, 0), (4, 1), (4, 2),
#     (4, 7), (4, 8), (4, 14), (5, 6), (5, 12), (5, 13), (5, 14), 
#     (6, 5), (6, 10), (7, 3),
# ]

for i in filled:
    xw[i] = BLACK
xw.fill(emordilaps_wordlist)